# Data Wrangling

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# acquire data from csv
jobs = pd.read_csv('fake_job_postings.csv')

In [3]:
# preview data
jobs.head(1)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0


In [4]:
# view info
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15185 non-null  object
 8   benefits             10670 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

None of the numerical columns are missing any values. It looks like the majority of object-type columns have some missing values. I want to investigate these columns further to see if I should clean them up a bit to use them in this project or if I can simply drop them.

From looking at the head of the dataframe, I can see that some of these columns don't bring much insight to the table and/or will be useless for the exploration and modeling I intend to do in this project so I plan to drop `job_id`, `title`, `company_profile`, `description`, `requirements`, and `benefits`.

In [5]:
# view null percentage by column
def cols_missing_rows(df):
    df = pd.DataFrame(data={'num_rows_missing':df.isnull().sum(), 
              'pct_rows_missing':df.isnull().sum()/len(df)}, index=df.columns)
    return df
cols_missing_rows(jobs.select_dtypes('O'))

,num_rows_missing,pct_rows_missing
title,0,0.000000
location,346,0.019351
department,11547,0.645805
salary_range,15012,0.839597
company_profile,3308,0.185011
description,1,0.000056
requirements,2695,0.150727
benefits,7210,0.403244
employment_type,3471,0.194128
required_experience,7050,0.394295


Looks like `department` and `salary_range` are missing the majority of their values. I will most likely drop these two columns completely since they are missing so much data.

For the columns missing less than 10% of their data I will just drop the rows containing null values since the only columns that fit this criteria are `location` (~2%) and `description` (missing only one value).

Next I want to further investigate those columns missing between 10 and 50 percent of their rows.

In [6]:
jobs[jobs.employment_type.isnull()]

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
5,6,Accounting Clerk,"US, MD,",NaN,NaN,NaN,Job OverviewApex is an environmental consultin...,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,0
7,8,Lead Guest Service Specialist,"US, CA, San Francisco",NaN,NaN,Airenvy’s mission is to provide lucrative yet ...,Who is Airenvy?Hey there! We are seasoned entr...,"Experience with CRM software, live chat, and p...",Competitive Pay. You'll be able to eat steak e...,0,1,1,NaN,NaN,NaN,NaN,NaN,0
11,12,Talent Sourcer (6 months fixed-term contract),"GB, LND, London",HR,NaN,Want to build a 21st century financial service...,TransferWise is the clever new way to move mon...,We’re looking for someone who:Proven track rec...,You will join one of Europe’s most hotly tippe...,0,1,0,NaN,NaN,NaN,NaN,NaN,0
17,18,Southend-on-Sea Traineeships Under NAS 16-18 Y...,"GB, SOS, Southend-on-Sea",NaN,NaN,Established on the principles that full time e...,Government funding is only available for 16-18...,16-18 year olds only due to government funding...,Career prospects.,0,1,1,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17848,17849,SEO CONTENT WRITER,"GB, LND, london",SEO,NaN,NaN,A Blogger or Journalist is required for delive...,Key responsibilities within this role:Supporti...,We offer: • Excellent training and development...,0,0,1,NaN,NaN,NaN,NaN,NaN,0
17853,17854,Call Center/Customer Service,"US, NJ, Newark",Customer Service,NaN,At Command we care enough to consistently plac...,At Command we care enough to consistently plac...,Responsibilities:Determines requirements by wo...,Benefits:15/hr (non-negotiable)Medical/dental ...,0,1,1,NaN,NaN,NaN,NaN,NaN,0
17855,17856,Android Engineer,"US, CA, San Francisco",Engineering,NaN,"Shyp is the easiest way to send anything, anyw...",What we're looking for:You've been an integral...,Track record of developing native Android appl...,NaN,0,1,0,NaN,NaN,NaN,NaN,Engineering,0
17859,17860,Immediate Requirement: iOS Developer for Qatar,"QA, DA,",NaN,NaN,NaN,Greetings from VAM SYSTEMS…..VAM SYSTEMS is a ...,"iOS programming “Objective – C “, C++#URL_01a...",NaN,0,0,1,NaN,NaN,NaN,NaN,NaN,0


It looks like a lot of these rows are missing several columns of data. I'll look into this a little further:

In [7]:
# view missing data by rows
def rows_missing_cols(df):
    df = pd.DataFrame({'num_cols_missing':df.isnull().sum(axis=1).value_counts().index,
                       'pct_cols_missing':df.isnull().sum(axis=1).value_counts().index/len(df.columns),
                       'num_rows':df.isnull().sum(axis=1).value_counts()}).reset_index(drop=True)
    return df
rows_missing_cols(jobs)

,num_cols_missing,pct_cols_missing,num_rows
0,2,0.111111,3314
1,3,0.166667,3091
2,1,0.055556,2309
3,4,0.222222,2205
4,7,0.388889,1421
5,6,0.333333,1378
6,8,0.444444,1294
7,5,0.277778,1169
8,0,0.000000,774
9,9,0.500000,602


In [8]:
# view value counts
jobs.required_education.value_counts(dropna=False)

NaN                                  8105
Bachelor's Degree                    5145
High School or equivalent            2080
Unspecified                          1397
Master's Degree                       416
Associate Degree                      274
Certification                         170
Some College Coursework Completed     102
Professional                           74
Vocational                             49
Some High School Coursework            27
Doctorate                              26
Vocational - HS Diploma                 9
Vocational - Degree                     6
Name: required_education, dtype: int64

I want to combine these categories into broader categories: None, College, and Vocational. I will treat the null values as 'None'.

- None: NaN, High School or equivalent, Unspecified, Some College Coursework Completed, Some High School Coursework
- College: Bachelor's Degree, Master's Degree, Associate Degree, Doctorate
- Vocational: Certification, Vocational, Vocational - HS Diploma, Vocational - Degree, Professional

In [9]:
# create copy df
jobs1 = jobs.copy()
# define function to categorize education reqs
def fix_edu(jobs1):
    jobs1.required_education = np.where(jobs1.required_education.isna(), 'None', jobs1.required_education)
    jobs1.required_education = np.where(jobs1.required_education.isin(['High School or equivalent', 'Unspecified','Some College Coursework Completed', 'Some High School Coursework']), 'None', jobs1.required_education)
    jobs1.required_education = np.where(jobs1.required_education.isin(['Bachelor\'s Degree', 'Master\'s Degree', 'Associate Degree', 'Doctorate']), 'College', jobs1.required_education)
    jobs1.required_education = np.where(jobs1.required_education.isin(['Certification', 'Vocational', 'Vocational - HS Diploma', 'Vocational - Degree', 'Professional']), 'Vocational', jobs1.required_education)
    return jobs1
# test function
fix_edu(jobs1).required_education.value_counts(dropna=False)

None          11711
College        5861
Vocational      308
Name: required_education, dtype: int64

In [10]:
# view value counts
jobs.required_experience.value_counts(dropna=False)

NaN                 7050
Mid-Senior level    3809
Entry level         2697
Associate           2297
Not Applicable      1116
Director             389
Internship           381
Executive            141
Name: required_experience, dtype: int64

For these null values, I will treat them the same as 'Not Applicable'. I will also be combining these values into broader categories: Low, Medium, High.

- Low: NaN, Entry level, Associate, Not Applicable, Internship
- Med: Mid-Senior Level
- High: Director, Executive

In [11]:
# define function to categorize experience reqs
def fix_exp(jobs1):
    jobs1.required_experience = np.where(jobs1.required_experience.isna(), 'Low', jobs1.required_experience)
    jobs1.required_experience = np.where(jobs1.required_experience.isin(['Entry level', 'Associate', 'Not Applicable', 'Internship']), 'Low', jobs1.required_experience)
    jobs1.required_experience = np.where(jobs1.required_experience == 'Mid-Senior level', 'Med', jobs1.required_experience)
    jobs1.required_experience = np.where(jobs1.required_experience.isin(['Director', 'Executive']), 'High', jobs1.required_experience)
    return jobs1
# test function
fix_exp(jobs1).required_experience.value_counts(dropna=False)

Low     13541
Med      3809
High      530
Name: required_experience, dtype: int64

In [12]:
# view employment_type value counts
jobs.employment_type.value_counts(dropna=False)

Full-time    11620
NaN           3471
Contract      1524
Part-time      797
Temporary      241
Other          227
Name: employment_type, dtype: int64

For this column, I don't feel comfortable filling the null values with any of the other employment types, and I do not want to drop all of these nulls because they comprise nearly 20% of the data. I will instead create categories like I did for experience and education. These will include:
- Employee: Full-time, Part-time
- Non-employee: Contract, Temporary
- Unspecified: NaN, Other

In [13]:
# define function to categorize employment types
def fix_emp(jobs1):
    jobs1.employment_type = np.where(jobs1.employment_type.isna(), 'Unspecified', jobs1.employment_type)
    jobs1.employment_type = np.where(jobs1.employment_type == 'Other', 'Unspecified', jobs1.employment_type)
    jobs1.employment_type = np.where(jobs1.employment_type.isin(['Full-time', 'Part-time']), 'Employee', jobs1.employment_type)
    jobs1.employment_type = np.where(jobs1.employment_type.isin(['Contract', 'Temporary']), 'Non-employee', jobs1.employment_type)
    return jobs1
# test function
fix_emp(jobs1).employment_type.value_counts(dropna=False)

Employee        12417
Unspecified      3698
Non-employee     1765
Name: employment_type, dtype: int64

In [14]:
# view industry value counts
jobs.industry.value_counts(dropna=False)

NaN                                    4903
Information Technology and Services    1734
Computer Software                      1376
Internet                               1062
Marketing and Advertising               828
                                       ... 
Museums and Institutions                  1
Ranching                                  1
Shipbuilding                              1
Alternative Dispute Resolution            1
Wine and Spirits                          1
Name: industry, Length: 132, dtype: int64

In [15]:
# number of industries
jobs.industry.nunique(dropna=False)

132

In [16]:
# view value counts for functions
jobs.function.value_counts(dropna=False)

NaN                       6455
Information Technology    1749
Sales                     1468
Engineering               1348
Customer Service          1229
Marketing                  830
Administrative             630
Design                     340
Health Care Provider       338
Education                  325
Other                      325
Management                 317
Business Development       228
Accounting/Auditing        212
Human Resources            205
Project Management         183
Finance                    172
Consulting                 144
Writing/Editing            132
Art/Creative               132
Production                 116
Product Management         114
Quality Assurance          111
Advertising                 90
Business Analyst            84
Data Analyst                82
Public Relations            76
Manufacturing               74
General Business            68
Research                    50
Legal                       47
Strategy/Planning           46
Training

In [17]:
# number of function values
jobs.function.nunique(dropna=False)

38

In the interest of time, I will drop `industry` and `function` for now as there are 132 unique industry values and 38 unique function values. I don't want to overload my model with too many features to begin with so for now I will put these aside. If I have time in the future, I would like to either categorize these as I did with other columns or focus on exploring these columns individually.

For this project, the columns I will be using include:
- location
- telecommuting
- has_company_logo
- has_questions
- employment_type
- required_experience
- required_education
- fraudulent (target variable)

In [18]:
# list columns to be dropped
cols_to_drop = ['job_id', 'title', 'company_profile', 'description', 'requirements', 'benefits', 'department', 'salary_range', 'industry', 'function']

# drop columns
jobs_proj = jobs.drop(columns=cols_to_drop)
jobs_proj

,location,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent
0,"US, NY, New York",0,1,0,Other,Internship,NaN,0
1,"NZ, , Auckland",0,1,0,Full-time,Not Applicable,NaN,0
2,"US, IA, Wever",0,1,0,NaN,NaN,NaN,0
3,"US, DC, Washington",0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,0
4,"US, FL, Fort Worth",0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,0
...,...,...,...,...,...,...,...,...
17875,"CA, ON, Toronto",0,1,1,Full-time,Mid-Senior level,NaN,0
17876,"US, PA, Philadelphia",0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,0
17877,"US, TX, Houston",0,0,0,Full-time,NaN,NaN,0
17878,"NG, LA, Lagos",0,0,1,Contract,Not Applicable,Professional,0


In [19]:
# clean columns using functions
jobs_proj = fix_emp(fix_exp(fix_edu(jobs_proj)))
jobs_proj

,location,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent
0,"US, NY, New York",0,1,0,Unspecified,Low,None,0
1,"NZ, , Auckland",0,1,0,Employee,Low,None,0
2,"US, IA, Wever",0,1,0,Unspecified,Low,None,0
3,"US, DC, Washington",0,1,0,Employee,Med,College,0
4,"US, FL, Fort Worth",0,1,1,Employee,Med,College,0
...,...,...,...,...,...,...,...,...
17875,"CA, ON, Toronto",0,1,1,Employee,Med,None,0
17876,"US, PA, Philadelphia",0,1,1,Employee,Med,College,0
17877,"US, TX, Houston",0,0,0,Employee,Low,None,0
17878,"NG, LA, Lagos",0,0,1,Non-employee,Low,Vocational,0


In [20]:
# view remaining nulls
cols_missing_rows(jobs_proj)

,num_rows_missing,pct_rows_missing
location,346,0.019351
telecommuting,0,0.000000
has_company_logo,0,0.000000
has_questions,0,0.000000
employment_type,0,0.000000
required_experience,0,0.000000
required_education,0,0.000000
fraudulent,0,0.000000


In [21]:
# drop remaining nulls
jobs_proj = jobs_proj.dropna()
cols_missing_rows(jobs_proj)

,num_rows_missing,pct_rows_missing
location,0,0.0
telecommuting,0,0.0
has_company_logo,0,0.0
has_questions,0,0.0
employment_type,0,0.0
required_experience,0,0.0
required_education,0,0.0
fraudulent,0,0.0


In [22]:
# view dataframe
jobs_proj

,location,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent
0,"US, NY, New York",0,1,0,Unspecified,Low,None,0
1,"NZ, , Auckland",0,1,0,Employee,Low,None,0
2,"US, IA, Wever",0,1,0,Unspecified,Low,None,0
3,"US, DC, Washington",0,1,0,Employee,Med,College,0
4,"US, FL, Fort Worth",0,1,1,Employee,Med,College,0
...,...,...,...,...,...,...,...,...
17875,"CA, ON, Toronto",0,1,1,Employee,Med,None,0
17876,"US, PA, Philadelphia",0,1,1,Employee,Med,College,0
17877,"US, TX, Houston",0,0,0,Employee,Low,None,0
17878,"NG, LA, Lagos",0,0,1,Non-employee,Low,Vocational,0


In [23]:
# view unique job locations
jobs.location.nunique()

3105

In [24]:
# view unique countries
jobs.location.str[:2].value_counts()

US    10656
GB     2384
GR      940
CA      457
DE      383
      ...  
SV        1
CM        1
CO        1
SI        1
SD        1
Name: location, Length: 90, dtype: int64

In [25]:
jobs_proj['country'] = jobs.location.str[:2]
jobs_proj

<ipython-input-25-1266f4e7214e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_proj['country'] = jobs.location.str[:2]


,location,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent,country
0,"US, NY, New York",0,1,0,Unspecified,Low,None,0,US
1,"NZ, , Auckland",0,1,0,Employee,Low,None,0,NZ
2,"US, IA, Wever",0,1,0,Unspecified,Low,None,0,US
3,"US, DC, Washington",0,1,0,Employee,Med,College,0,US
4,"US, FL, Fort Worth",0,1,1,Employee,Med,College,0,US
...,...,...,...,...,...,...,...,...,...
17875,"CA, ON, Toronto",0,1,1,Employee,Med,None,0,CA
17876,"US, PA, Philadelphia",0,1,1,Employee,Med,College,0,US
17877,"US, TX, Houston",0,0,0,Employee,Low,None,0,US
17878,"NG, LA, Lagos",0,0,1,Non-employee,Low,Vocational,0,NG


Using the country column I created, I would like to categorize the 90 countries in this dataset by their continents.
- NA = North America
- SA = South America
- AF = Africa
- AN = Antarctica
- AS = Asia
- EU = Europe
- OC = Oceania

In [26]:
# group countries into continents

import pycountry_convert as pc
jobs_proj['continent'] = [pc.country_alpha2_to_continent_code(loc) for loc in jobs_proj.country]
jobs_proj.continent.value_counts()

<ipython-input-26-2e70fdfd2d4e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_proj['continent'] = [pc.country_alpha2_to_continent_code(loc) for loc in jobs_proj.country]


NA    11153
EU     4745
AS      908
OC      547
AF      132
SA       49
Name: continent, dtype: int64

In [27]:
[pc.country_alpha2_to_continent_code(loc) for loc in jobs_proj.location.str[:2]]

['NA',
 'OC',
 'NA',
 'NA',
 'NA',
 'NA',
 'EU',
 'NA',
 'NA',
 'NA',
 'NA',
 'EU',
 'NA',
 'NA',
 'OC',
 'AS',
 'AS',
 'EU',
 'NA',
 'NA',
 'NA',
 'OC',
 'AS',
 'NA',
 'EU',
 'NA',
 'AS',
 'AS',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'EU',
 'NA',
 'NA',
 'NA',
 'AS',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'AF',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'EU',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'OC',
 'EU',
 'NA',
 'NA',
 'NA',
 'EU',
 'NA',
 'NA',
 'AS',
 'NA',
 'EU',
 'NA',
 'NA',
 'NA',
 'OC',
 'NA',
 'NA',
 'EU',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'EU',
 'NA',
 'NA',
 'EU',
 'NA',
 'NA',
 'EU',
 'NA',
 'EU',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'EU',
 'NA',
 'NA',
 'NA',
 'EU',
 'EU',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'EU',
 'NA',
 'EU',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'EU',
 'NA',
 'NA',
 'NA',
 'NA',
 'EU',
 'NA',
 'NA',
 'NA',
 'NA',
 'EU',
 'EU',
 'NA',
 'EU',
 'NA',
 'NA',
 'EU',
 'NA',
 'NA',
 'NA',
 'NA',
 'AS',

In [28]:
# combine previous steps into wrangle function
def wrangle_jobs():
    '''
    This function takes in no arguments; it acquires and prepares job listing data
    from a local csv file named fake_job_postings.csv.
    '''
    jobs = pd.read_csv('fake_job_postings.csv')
    cols_to_drop = ['job_id', 'title', 'company_profile', 'description', 'requirements', 'benefits', 'department', 'salary_range', 'industry', 'function']
    jobs = jobs.drop(columns=cols_to_drop)
    jobs = fix_emp(fix_exp(fix_edu(jobs)))
    jobs = jobs_proj.dropna()
    jobs['continent'] = [pc.country_alpha2_to_continent_code(loc) for loc in jobs_proj.location.str[:2]]
    jobs.drop(columns='location', inplace=True)
    return jobs
# test function
wrangle_jobs()

,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent,country,continent
0,0,1,0,Unspecified,Low,None,0,US,NA
1,0,1,0,Employee,Low,None,0,NZ,OC
2,0,1,0,Unspecified,Low,None,0,US,NA
3,0,1,0,Employee,Med,College,0,US,NA
4,0,1,1,Employee,Med,College,0,US,NA
...,...,...,...,...,...,...,...,...,...
17875,0,1,1,Employee,Med,None,0,CA,NA
17876,0,1,1,Employee,Med,College,0,US,NA
17877,0,0,0,Employee,Low,None,0,US,NA
17878,0,0,1,Non-employee,Low,Vocational,0,NG,AF


In [29]:
# test function from module
import wrangle as w
w.wrangle_jobs()

,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent,continent
0,0,1,0,Unspecified,Low,None,0,NA
1,0,1,0,Employee,Low,None,0,OC
2,0,1,0,Unspecified,Low,None,0,NA
3,0,1,0,Employee,Med,College,0,NA
4,0,1,1,Employee,Med,College,0,NA
...,...,...,...,...,...,...,...,...
17875,0,1,1,Employee,Med,None,0,NA
17876,0,1,1,Employee,Med,College,0,NA
17877,0,0,0,Employee,Low,None,0,NA
17878,0,0,1,Non-employee,Low,Vocational,0,AF
